In [11]:
import shutil
import os

# Set working directory one path up (to root of the repo)
os.chdir('..')

path_to_dataset = 'datasets/colorful_fashion'

CLASS_MAPPING = {
    "0": "sunglass",
    "1": "hat",
    "2": "jacket",
    "3": "shirt",
    "4": "pants",
    "5": "shorts",
    "6": "skirt",
    "7": "dress",
    "8": "bag",
    "9": "shoe"
    }

### Data preparation

In [7]:
# Create or empty folders to store the yolo dataset
for set_type in ['train', 'val']:
    for file_type in ['images', 'labels']:
        # Create directory when it does not exist
        if not os.path.exists(f'{path_to_dataset}/yolo/{set_type}/{file_type}'):
            os.makedirs(f'{path_to_dataset}/yolo/{set_type}/{file_type}')
        # Empty folder if it already exists
        else:
            for file in os.scandir(f'{path_to_dataset}/yolo/{set_type}/{file_type}'):
                os.remove(file.path)

# Create train set
with open(f'{path_to_dataset}/ImageSets/Main/trainval.txt', 'r') as file:
    for line in file:
        name = line.strip()  # removes the new line character

        # Copy image to train set
        shutil.copyfile(f'{path_to_dataset}/JPEGImages/{name}.jpg', f'../datasets/colorful_fashion/yolo/train/images/{name}.jpg')    

        # Copy label to train set
        shutil.copyfile(f'{path_to_dataset}/Annotations_txt/{name}.txt', f'../datasets/colorful_fashion/yolo/train/labels/{name}.txt')

# Create val set
with open(f'{path_to_dataset}/ImageSets/Main/test.txt', 'r') as file:
    for line in file:
        name = line.strip()  # removes the new line character

        # Copy image to val set
        shutil.copyfile(f'{path_to_dataset}/JPEGImages/{name}.jpg', f'../datasets/colorful_fashion/yolo/val/images/{name}.jpg')    

        # Copy label to val set
        shutil.copyfile(f'{path_to_dataset}/Annotations_txt/{name}.txt', f'../datasets/colorful_fashion/yolo/val/labels/{name}.txt')


### Data exploration

In [13]:
from utils.data_exploration import get_image_labels_local, print_dataset_statistics, plot_yolo_labels

In [14]:
image_labels_train = get_image_labels_local(yolo_folder=f'{path_to_dataset}/yolo', dataset_type='train')
image_labels_val = get_image_labels_local(yolo_folder=f'{path_to_dataset}/yolo', dataset_type='val')

In [15]:
print_dataset_statistics(image_labels_train, 'train', CLASS_MAPPING)
print_dataset_statistics(image_labels_val, 'val', CLASS_MAPPING)

Train set consists of 2145 images
The labels are distributed as follows:
 shoe        2081
shirt       1486
bag         1188
jacket       777
skirt        697
dress        550
pants        508
shorts       395
sunglass     343
hat          273
Name: count, dtype: int64
Val set consists of 537 images
The labels are distributed as follows:
 shoe        520
shirt       366
bag         274
skirt       186
jacket      181
dress       128
pants       114
shorts      107
sunglass     82
hat          77
Name: count, dtype: int64


In [16]:
plot_yolo_labels(yolo_folder=f'{path_to_dataset}/yolo', yolo_plotting_folder=f'{path_to_dataset}/yolo_plotting')

### Model training

In [17]:
import yaml

# Define variables for data config yaml
path = '../../input/data'
images_path_train = 'train/images'
images_path_val = 'val/images'

class_count = len(CLASS_MAPPING)
class_names = list(CLASS_MAPPING.values())

data = {
    'path': path,
    'train': images_path_train,
    'val': images_path_val,
    'nc': class_count,
    'names': class_names,
}

path_to_yml = 'yolov8/data.yaml'


# Create/update yaml containing the data config
with open(path_to_yml, 'w') as outfile:
    yaml.dump(data, outfile, sort_keys=False, default_flow_style=None)

In [ ]:
import sagemaker
from sagemaker import get_execution_role
from sagemaker.pytorch import PyTorch
from datetime import datetime

# Job name
now = datetime.now()
job_name = f'yolov8-{JOB_NAME}-{now.strftime("%Y-%m-%d-%H-%M-%S")}'

# Paths to datasets
s3_path_dataset_train = f's3://{S3_BUCKET_INPUT}/{S3_PREFIX_YOLO_DATASET}/train/'
s3_path_dataset_val = f's3://{S3_BUCKET_INPUT}/{S3_PREFIX_YOLO_DATASET}/val/'

# Metrics for training
metric_definitions = [{'Name': 'Epoch_best', 'Regex': 'Epoch_best=(.*?);'},
                      {'Name': 'Precision', 'Regex': 'Precision=(.*?);'},
                      {'Name': 'Recall', 'Regex': 'Recall=(.*?);'},
                      {'Name': 'mAP@.5', 'Regex': 'mAP@.5=(.*?);'},
                      {'Name': 'mAP@.5:.95', 'Regex': 'mAP@.5:.95=(.*?);'}]

# Datasets used during training 
inputs = {'train': s3_path_dataset_train, 'val': s3_path_dataset_val}

# Define training settings
pytorch_estimator = PyTorch(
                    instance_type='ml.p3.2xlarge', # ml.p3.2xlarge or ml.p3.8xlarge (gpu) or ml.c5.9xlarge (cpu)
                    instance_count=1,
                    entry_point="train.py",
                    source_dir="yolov8",
                    role = 'arn:aws:iam::020223827588:role/model-training-cv-playground-sagemaker-execution', #get_execution_role(),
                    job_name = job_name,
                    checkpoint_s3_uri = f's3://model-training-cv-playground-model-artifact/YOLOV8/{job_name}/checkpoints',
                    use_spot_instances = True,
                    max_wait = 14800, # 4hours
                    max_run = 7200, # 2hours
                    image_uri = '020223827588.dkr.ecr.eu-west-1.amazonaws.com/ultralytics-yolov8-sagemaker:latest',
                    hyperparameters = {'training-job-name': job_name,
                                       # For all YOLOV8 default parameters, check https://github.com/ultralytics/ultralytics/blob/790f9c067c9a3548acf7c6925cea14de9ad77787/ultralytics/yolo/cfg/default.yaml
                                       # Train settings --------------------------------------------------------------
                                       'base_model': 'yolov8l.pt', # path to model file, i.e. yolov8n.pt, yolov8n.yaml
                                       'data': 'data.yaml', # path to data file, i.e. coco128.yaml
                                       'epochs': 20, # number of epochs to train for
                                       'patience': 20, # epochs to wait for no observable improvement for early stopping of training
                                       'batch': -1, # number of images per batch (-1 for AutoBatch)
                                       'imgsz': 1280, # size of input images as integer or w,h
                                       'save_period': 10, # Save checkpoint every x epochs (disabled if < 1)
                                       'image_weights': False, # use weighted image selection for training
                                       'rect': False, # support rectangular training if mode='train', support rectangular evaluation if mode='val'
                                       'close_mosaic': 10, # disable mosaic augmentation for final 10 epochs
                                       'lr0': 0.01, # initial learning rate (i.e. SGD=1E-2, Adam=1E-3)
                                       'lrf': 0.01, # final learning rate (lr0 * lrf)
                                       'dropout': 0.0,
                                       # Hyper parameters for data augmentation --------------------------------------
                                       'degrees': 0.0, # image rotation (+/- deg)
                                       'translate': 0.1,  # image translation (+/- fraction)
                                       'scale': 0.5,  # image scale (+/- gain)
                                       'shear': 0.0,  # image shear (+/- deg)
                                       'perspective': 0.0,  # image perspective (+/- fraction), range 0-0.001
                                       'flipud': 0.0,  # image flip up-down (probability)
                                       'fliplr': 0.5,  # image flip left-right (probability)
                                       'mosaic': 1.0,  # image mosaic (probability)
                                       'mixup': 0.0,  # image mixup (probability)
                                       'copy_paste': 0.1  # segment copy-paste (probability)
                                      },
                    metric_definitions=metric_definitions)

# Start a SageMaker training job
pytorch_estimator.fit(inputs=inputs)

### Hyperparameter tuning

In [ ]:
# from sagemaker.tuner import (
#     IntegerParameter,
#     CategoricalParameter,
#     ContinuousParameter,
#     HyperparameterTuner,
# )

# hyperparameter_ranges = {
#     "lr0": ContinuousParameter(0.0001, 0.01),
#     "dropout": ContinuousParameter(0.0, 0.4)
#     }


# tuner = HyperparameterTuner(
#     estimator =pytorch_estimator,
#     objective_metric_name='mAP@.5:.95',
#     hyperparameter_ranges=hyperparameter_ranges,
#     metric_definitions=metric_definitions,
#     strategy='Bayesian',
#     objective_type='Maximize',
#     max_jobs=6,
#     max_parallel_jobs=1)

# tuner.fit(inputs=inputs)

### Test model

In [ ]:
import boto3

s3_client = boto3.client('s3')

# Download model from S3
s3_client.download_file('model-training-cv-playground-model-artifact', 'YOLOV8/yolov8-soccer-player-detection-2023-05-03-16-44-18/checkpoints/weights/best.pt', 'best_soccer_player.pt')

# Download test image from S3
s3_client.download_file('model-training-cv-playground-model-input', 'datasets/soccer_player/test/images/1-fps-2_00001_jpeg_jpg.rf.03aa7dfbdbc3d0a5482ea68a7f9a8a8d.jpg', '1-fps-2_00001_jpeg_jpg.rf.03aa7dfbdbc3d0a5482ea68a7f9a8a8d.jpg')

In [ ]:
from ultralytics import YOLO

# Load a model
model = YOLO('best_soccer_player.pt')

# Perform inference on test image
results = model('1-fps-2_00001_jpeg_jpg.rf.03aa7dfbdbc3d0a5482ea68a7f9a8a8d.jpg')

In [ ]:
import cv2
from ultralytics.yolo.utils.plotting import Annotator

img = cv2.imread('1-fps-2_00001_jpeg_jpg.rf.03aa7dfbdbc3d0a5482ea68a7f9a8a8d.jpg', cv2.COLOR_BGR2RGB)

annotator = Annotator(img)

for box in results[0].boxes:
    
    if float(box.conf) >0.5:
        b = box.xyxy[0]  # get box coordinates in (top, left, bottom, right) format
        c = box.cls
        annotator.box_label(b, model.names[int(c)])
        
cv2.imwrite('output.png', img)     